### Chains with multiple inputs

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain

llm = ChatOpenAI(model="gpt-4o-mini")

prompt_template = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input="a parrot")

/var/folders/d1/d31w101x0f31vstymvwz8kwr0000gn/T/ipykernel_19411/2486307642.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


{'input': 'a parrot',
 'text': 'Why did the parrot wear a raincoat?\n\nBecause it wanted to be a polyunsaturated!'}

In [3]:
prompt_template = PromptTemplate(input_variables=["input", "language"], template="Tell me a joke about {input} in {language}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke({"input": "a parrot", "language": "Kannada"})

{'input': 'a parrot',
 'language': 'Kannada',
 'text': 'ಹೌದು, ಇಲ್ಲಿದೆ ಒಂದು ಪಾರಟ್ ಬಗ್ಗೆ ಚುಟುಕುವಾಸ್ತವ:\n\nಪಾರ್ಟ್: "ನಾನು ಮಾತನಾಡಬಹುದು!"\nತಿಮ್ಮಪ್ಪ: "ಅರ್ಥವಾಗಿದೆ, ಆದರೆ ನನ್ನ ಹೆಂಡತಿ ಮಾತಾಡುತ್ತಿದ್ದಾಗ ನಿನ್ನ ಕೇಳುವುದಿಲ್ಲ!"\n\n(ಇದು ಪಾರ್ಟ್ ಮತ್ತು ತಿಮ್ಮಪ್ಪನ ನಡುವೆ ನಡೆದ ಮಾತುಕತೆ, ಹಾಸ್ಯವನ್ನು ತರುವಂತೆ! :) )'}

Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [4]:
from langchain.chains.sequential import SequentialChain

# This is an LLMChain to write a review given a dish name and the experience.
prompt_review = PromptTemplate.from_template(
    template="You ordered {dish_name} and your experience was {experience}. Write a review: "
)
chain_review = LLMChain(llm=llm, prompt=prompt_review, output_key="review")

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = PromptTemplate.from_template(
    template="Given the restaurant review: {review}, write a follow-up comment: "
)
chain_comment = LLMChain(llm=llm, prompt=prompt_comment, output_key="comment")

# This is an LLMChain to summarize a review.
prompt_summary = PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {comment}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")

# This is an LLMChain to translate a summary into German.
prompt_translation = PromptTemplate.from_template(
    template="Translate the summary to Hindi: \n\n {summary}"
)
chain_translation = LLMChain(
    llm=llm, prompt=prompt_translation, output_key="hindi_translation"
)

In [5]:
overall_chain = SequentialChain(
    chains=[chain_review, chain_comment, chain_summary, chain_translation],
    input_variables=["dish_name", "experience"],
    output_variables=["review", "comment", "summary", "hindi_translation"],
)

overall_chain.invoke({"dish_name": "Pizza Salami", "experience": "It was awful!"})

{'dish_name': 'Pizza Salami',
 'experience': 'It was awful!',
 'review': '**Review of Pizza Salami: A Disappointing Experience**\n\nI recently ordered a Pizza Salami, hoping for a delicious meal, but unfortunately, my experience was far from satisfactory. \n\nFirst off, the pizza arrived later than expected. I understand that sometimes delays happen, but it would have been nice to receive an update or estimate on the delivery time. When it finally arrived, I was met with a soggy crust that reminded me more of cardboard than the crispy base I was hoping for. \n\nThe salami itself was a letdown as well. Instead of being flavorful and satisfying, it lacked any real taste and seemed stale. The balance of cheese and sauce was off; there was far too much sauce, making the entire pizza overly soggy and difficult to eat. I ended up picking off some of the toppings just to salvage the experience. \n\nLastly, the overall presentation was also disappointing. A pizza should be inviting, but this l

### Instead of chaining multiple chains together we can also use an LLM to decide which follow up chain is being used

In [6]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

positive_template = """You are an AI that focuses on the positive side of things. \
Whenever you analyze a text, you look for the positive aspects and highlight them. \
Here is the text:
{input}"""

neutral_template = """You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, \
not favoring any positive or negative aspects. Here is the text:
{input}"""

negative_template = """You are an AI that is designed to find the negative aspects in a text. \
You analyze a text and show the potential downsides. Here is the text:
{input}"""

In [7]:
prompt_infos = [
    {
        "name": "positive",
        "description": "Good for analyzing positive sentiments",
        "prompt_template": positive_template,
    },
    {
        "name": "neutral",
        "description": "Good for analyzing neutral sentiments",
        "prompt_template": neutral_template,
    },
    {
        "name": "negative",
        "description": "Good for analyzing negative sentiments",
        "prompt_template": negative_template,
    },
]

In [8]:
destination_chains = {}
for p_info in prompt_infos:
    # Here we are creating a dictionary of chains, where the key is the name of the chain and the value is the chain itself.
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
destination_chains

{'positive': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an AI that focuses on the positive side of things. Whenever you analyze a text, you look for the positive aspects and highlight them. Here is the text:\n{input}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11b0c5a50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11b5724d0>, root_client=<openai.OpenAI object at 0x11ad4ca50>, root_async_client=<openai.AsyncOpenAI object at 0x11b571e50>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}),
 'neutral': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, not favoring 

In [9]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

positive: Good for analyzing positive sentiments
neutral: Good for analyzing neutral sentiments
negative: Good for analyzing negative sentiments


In [10]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(), # RouterOutputParser is used to parse the output of the router chain into a dictionary
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=destination_chains["neutral"],
    verbose=True,
)

chain.invoke({"input": "I ordered Pizza Salami for 9.99$ and it was not good!"})



> Entering new MultiPromptChain chain...


/var/folders/d1/d31w101x0f31vstymvwz8kwr0000gn/T/ipykernel_19411/731848241.py:10: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


negative: {'input': 'I ordered Pizza Salami for $9.99 and it was not good!'}
> Finished chain.


{'input': 'I ordered Pizza Salami for $9.99 and it was not good!',
 'text': 'Here are the potential downsides based on the provided text:\n\n1. **Quality Issues**: The pizza did not meet expectations, indicating poor quality or taste, which can lead to dissatisfaction with the product.\n\n2. **Value for Money**: Despite the price being relatively low at $9.99, the negative experience implies that the value received did not justify the cost.\n\n3. **Wasted Money**: Purchasing something that was not good can feel like a waste of money, leading to frustration.\n\n4. **Disappointment**: The disappointment in the food can affect the overall dining experience, potentially leading to negative feelings about the occasion.\n\n5. **Loss of Trust**: If this was from a restaurant previously considered reliable, one bad experience might diminish trust in their offerings.\n\n6. **Potential Health Concerns**: If the pizza was subpar in quality, there could be concerns regarding food safety or hygiene